In [1]:
import sys
import os
import json
import psycopg2
from psycopg2 import sql
from typing import Dict, Any
sys.path.append('/home/jovyan/work')
import src.conect_sql as conect_sql

In [2]:
# Diretório onde estão localizados os arquivos JSON
json_dir = '/home/jovyan/work/data/Spotify_history/'

In [3]:
def insert_data(
        conn: psycopg2.extensions.connection, 
        data: Dict[str, Any],
        your_table_name: str) -> None:
    """
    Insere dados no banco de dados PostgreSQL.

    Args:
        conn (psycopg2.extensions.connection): Objeto de conexão com o banco de dados.
        data (Dict[str, Any]): Dicionário contendo os dados a serem inseridos.

    Retorna:
        None
    """
    with conn.cursor() as cur:
        insert_query = sql.SQL(f"""
            INSERT INTO {your_table_name} (ts, platform, ms_played, conn_country, ip_addr, 
                                         master_metadata_track_name, master_metadata_album_artist_name, 
                                         master_metadata_album_album_name, spotify_track_uri, episode_name, 
                                         episode_show_name, spotify_episode_uri, reason_start, reason_end, 
                                         shuffle, skipped, offline, offline_timestamp, incognito_mode) 
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, to_timestamp(%s), %s)
        """)
        
        cur.execute(insert_query, (
            data['ts'], data['platform'], data['ms_played'], data['conn_country'], data['ip_addr'], 
            data['master_metadata_track_name'], data['master_metadata_album_artist_name'], 
            data['master_metadata_album_album_name'], data['spotify_track_uri'], data['episode_name'], 
            data['episode_show_name'], data['spotify_episode_uri'], data['reason_start'], data['reason_end'], 
            data['shuffle'], data['skipped'], data['offline'], data['offline_timestamp'], data['incognito_mode']
        ))
        
        conn.commit()

def process_json_files() -> None:
    """
    Processa arquivos JSON e insere os dados no banco de dados PostgreSQL.

    Retorna:
        None
    """
    conn = psycopg2.connect(**conect_sql.db_config)
    
    try:
        for filename in os.listdir(json_dir):
            if filename.endswith('.json'):
                print(f'Inserindo dados do arquivo:{filename}')
                with open(os.path.join(json_dir, filename), 'r') as file:
                    json_data = json.load(file)
                    
                    # Assumindo que json_data é uma lista de dicionários
                    for dicionario in json_data:
                        insert_data(conn = conn, data = dicionario, your_table_name = 'spotify_history')
                print(f'Dados de {filename} inseridos')
                print('=====')
    finally:
        conn.close()


In [4]:
process_json_files()

Inserindo dados do arquivo:Streaming_History_Audio_2019-2020_1.json
Dados de Streaming_History_Audio_2019-2020_1.json inseridos
=====
Inserindo dados do arquivo:Streaming_History_Audio_2019_0.json
Dados de Streaming_History_Audio_2019_0.json inseridos
=====
Inserindo dados do arquivo:Streaming_History_Audio_2020-2021_2.json
Dados de Streaming_History_Audio_2020-2021_2.json inseridos
=====
Inserindo dados do arquivo:Streaming_History_Audio_2021-2024_3.json
Dados de Streaming_History_Audio_2021-2024_3.json inseridos
=====
Inserindo dados do arquivo:Streaming_History_Audio_2024_4.json
Dados de Streaming_History_Audio_2024_4.json inseridos
=====
Inserindo dados do arquivo:Streaming_History_Video_2020-2024.json
Dados de Streaming_History_Video_2020-2024.json inseridos
=====


In [5]:

# if __name__ == '__main__':
#     process_json_files()
